In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5;pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 12580, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 12580 (delta 0), reused 0 (delta 0), pack-reused 12579
Receiving objects: 100% (12580/12580), 12.48 MiB | 10.68 MiB/s, done.
Resolving deltas: 100% (8625/8625), done.
     |████████████████████████████████| 596 kB 34.8 MB/s 


In [2]:
import json
import os
import shutil

class COCO2YOLO:
  # 소스 이미지 디렉토리와 Json annotation 파일, 타겟 이미지 디렉토리, 타겟 annotation 디렉토리를 생성자로 입력 받음. 
  def __init__(self, src_img_dir, json_file, tgt_img_dir, tgt_anno_dir):
    self.json_file = json_file
    self.src_img_dir = src_img_dir
    self.tgt_img_dir = tgt_img_dir
    self.tgt_anno_dir = tgt_anno_dir
    # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성. 
    self._check_file_and_dir(json_file, tgt_img_dir, tgt_anno_dir)
    # json 파일을 메모리로 로딩. 
    self.labels = json.load(open(json_file, 'r', encoding='utf-8'))
    # category id와 이름을 매핑하지만, 실제 class id는 이를 적용하지 않고 별도 적용. 
    self.coco_id_name_map = self._categories()
    self.coco_name_list = list(self.coco_id_name_map.values())
    print("total images", len(self.labels['images']))
    print("total categories", len(self.labels['categories']))
    print("total labels", len(self.labels['annotations']))
  
  # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성. 
  def _check_file_and_dir(self, file_path, tgt_img_dir, tgt_anno_dir):
    if not os.path.exists(file_path):
        raise ValueError("file not found")
    if not os.path.exists(tgt_img_dir):
        os.makedirs(tgt_img_dir)
    if not os.path.exists(tgt_anno_dir):
        os.makedirs(tgt_anno_dir)

  # category id와 이름을 매핑하지만, 추후에 class 명만 활용. 
  def _categories(self):
    categories = {}
    for cls in self.labels['categories']:
        categories[cls['id']] = cls['name']
    return categories
  
  # annotation에서 모든 image의 파일명(절대 경로 아님)과 width, height 정보 저장. 
  def _load_images_info(self):
    images_info = {}
    for image in self.labels['images']:
        id = image['id']
        file_name = image['file_name']
        if file_name.find('\\') > -1:
            file_name = file_name[file_name.index('\\')+1:]
        w = image['width']
        h = image['height']
  
        images_info[id] = (file_name, w, h)

    return images_info

  # ms-coco의 bbox annotation은 yolo format으로 변환. 좌상단 x, y좌표, width, height 기반을 정규화된 center x,y 와 width, height로 변환. 
  def _bbox_2_yolo(self, bbox, img_w, img_h):
    # ms-coco는 좌상단 x, y좌표, width, height
    x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
    # center x좌표는 좌상단 x좌표에서 width의 절반을 더함. center y좌표는 좌상단 y좌표에서 height의 절반을 더함.  
    centerx = bbox[0] + w / 2
    centery = bbox[1] + h / 2
    # centerx, centery, width, height를 이미지의 width/height로 정규화. 
    dw = 1 / img_w
    dh = 1 / img_h
    centerx *= dw
    w *= dw
    centery *= dh
    h *= dh
    return centerx, centery, w, h
  
  # image와 annotation 정보를 기반으로 image명과 yolo annotation 정보 가공. 
  # 개별 image당 하나의 annotation 정보를 가지도록 변환. 
  def _convert_anno(self, images_info):
    anno_dict = dict()
    for anno in self.labels['annotations']:
      bbox = anno['bbox']
      image_id = anno['image_id']
      category_id = anno['category_id']

      image_info = images_info.get(image_id)
      image_name = image_info[0]
      img_w = image_info[1]
      img_h = image_info[2]
      yolo_box = self._bbox_2_yolo(bbox, img_w, img_h)

      anno_info = (image_name, category_id, yolo_box)
      anno_infos = anno_dict.get(image_id)
      if not anno_infos:
        anno_dict[image_id] = [anno_info]
      else:
        anno_infos.append(anno_info)
        anno_dict[image_id] = anno_infos
    return anno_dict

  # class 명을 파일로 저장하는 로직. 사용하지 않음. 
  def save_classes(self):
    sorted_classes = list(map(lambda x: x['name'], sorted(self.labels['categories'], key=lambda x: x['id'])))
    print('coco names', sorted_classes)
    with open('coco.names', 'w', encoding='utf-8') as f:
      for cls in sorted_classes:
          f.write(cls + '\n')
    f.close()
  # _convert_anno(images_info)로 만들어진 anno 정보를 개별 yolo anno txt 파일로 생성하는 로직. 
  # coco2yolo()에서 anno_dict = self._convert_anno(images_info)로 만들어진 anno_dict를 _save_txt()에 입력하여 파일 생성
  def _save_txt(self, anno_dict):
    # 개별 image별로 소스 image는 타겟이미지 디렉토리로 복사하고, 개별 annotation을 타겟 anno 디렉토리로 생성. 
    for k, v in anno_dict.items():
      # 소스와 타겟 파일의 절대 경로 생성. 
      src_img_filename = os.path.join(self.src_img_dir, v[0][0])
      # tgt_anno_filename = os.path.join(self.tgt_anno_dir,v[0][0].split(".")[0] + ".txt")
      tgt_anno_filename = os.path.join(self.tgt_anno_dir,v[0][0][:v[0][0].rfind('.')] + ".txt")
      #print('source image filename:', src_img_filename, 'target anno filename:', tgt_anno_filename)
      # 이미지 파일의 경우 타겟 디렉토리로 단순 복사. 
      shutil.copy(src_img_filename, self.tgt_img_dir)
      # 타겟 annotation 출력 파일명으로 classid, bbox 좌표를 object 별로 생성. 
      with open(tgt_anno_filename, 'w', encoding='utf-8') as f:
        #print(k, v)
        # 여러개의 object 별로 classid와 bbox 좌표를 생성. 
        for obj in v:
          cat_name = self.coco_id_name_map.get(obj[1])
          # category_id는 class 명에 따라 0부터 순차적으로 부여. 
          category_id = self.coco_name_list.index(cat_name)
          #print('cat_name:', cat_name, 'category_id:', category_id)
          box = ['{:.6f}'.format(x) for x in obj[2]]
          box = ' '.join(box)
          line = str(category_id) + ' ' + box
          f.write(line + '\n')

  # ms-coco를 yolo format으로 변환. 
  def coco2yolo(self):
    print("loading image info...")
    images_info = self._load_images_info()
    print("loading done, total images", len(images_info))

    print("start converting...")
    anno_dict = self._convert_anno(images_info)
    print("converting done, total labels", len(anno_dict))

    print("saving txt file...")
    self._save_txt(anno_dict)
    print("saving done")

In [3]:
# 학습/검증/테스트용 images, labels 디렉토리 생성. 
!mkdir /content/cat;
!cd /content/cat; mkdir images; mkdir labels;
!cd /content/cat/images; mkdir train; mkdir val; mkdir test
!cd /content/cat/labels; mkdir train; mkdir val; mkdir test

In [4]:
# train 용 yolo 데이터 세트 생성. 
train_yolo_converter = COCO2YOLO(src_img_dir='/content/drive/MyDrive/Colab Notebooks/cat_dataset/train', json_file='/content/drive/MyDrive/Colab Notebooks/cat_dataset/train.json',
                                 tgt_img_dir='/content/cat/images/train', tgt_anno_dir='/content/cat/labels/train')
train_yolo_converter.coco2yolo()

# val 용 yolo 데이터 세트 생성. 
val_yolo_converter = COCO2YOLO(src_img_dir='/content/drive/MyDrive/Colab Notebooks/cat_dataset/valid', json_file='/content/drive/MyDrive/Colab Notebooks/cat_dataset/valid.json',
                                 tgt_img_dir='/content/cat/images/val', tgt_anno_dir='/content/cat/labels/val')
val_yolo_converter.coco2yolo()

# test 용 yolo 데이터 세트 생성. 
test_yolo_converter = COCO2YOLO(src_img_dir='/content/drive/MyDrive/Colab Notebooks/cat_dataset/test', json_file='/content/drive/MyDrive/Colab Notebooks/cat_dataset/test.json',
                                 tgt_img_dir='/content/cat/images/test', tgt_anno_dir='/content/cat/labels/test')
test_yolo_converter.coco2yolo()

total images 8298
total categories 3
total labels 9277
loading image info...
loading done, total images 8298
start converting...
converting done, total labels 6631
saving txt file...
saving done
total images 2363
total categories 3
total labels 2603
loading image info...
loading done, total images 2363
start converting...
converting done, total labels 1878
saving txt file...
saving done
total images 1180
total categories 3
total labels 1351
loading image info...
loading done, total images 1180
start converting...
converting done, total labels 958
saving txt file...
saving done


In [5]:
!mkdir "/content/ultra_workdir"

In [8]:
!cd /content/yolov5; python train.py --img 640 --batch 8 --epochs 30 --data /content/cat/cat.yaml --weights yolov5l.pt \
                                     --project=/content/ultra_workdir --name cat --exist-ok 

train: weights=yolov5l.pt, cfg=, data=/content/cat/cat.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/ultra_workdir, name=cat, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-380-gb551098 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hs

In [20]:
import shutil

shutil.copy2("/content/cat/cat.yaml", "/content/drive/MyDrive/Colab Notebooks/cat_dataset/ultra_workdir_yolov5/cat.yaml")
shutil.copy2("/content/ultra_workdir/cat/weights/best.pt", "/content/drive/MyDrive/Colab Notebooks/cat_dataset/ultra_workdir_yolov5/best.pt")

'/content/drive/MyDrive/Colab Notebooks/cat_dataset/ultra_workdir_yolov5/cat.yaml'

In [22]:
!cd /content/yolov5;python detect.py --source /content/cat1.mp4 \
                            --weights /content/ultra_workdir/cat/weights/best.pt --conf 0.01 \
                            --project=/content/drive/MyDrive/Colab\ Notebooks/cat_dataset/ultra_workdir_yolov5/output --name=run_image --exist-ok --line-thickness 2

detect: weights=['/content/ultra_workdir/cat/weights/best.pt'], source=/content/cat1.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.01, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/Colab Notebooks/cat_dataset/ultra_workdir_yolov5/output, name=run_image, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-380-gb551098 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 367 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs
video 1/1 (1/1671) /content/cat1.mp4: 384x640 1 Cat___, Done. (0.035s)
video 1/1 (2/1671) /content/cat1.mp4: 384x640 Done. (0.034s)
video 1/1 (3/1671) /content/cat1.mp4: 384x640 Done. (0.024s)
video 1/1 (4/1671) /content/cat1.mp4: 384x640 Done. (0.024s)
video 1/1 (5/1671) /co

In [18]:
!cd /content/yolov5; python val.py --weights /content/ultra_workdir/cat/weights/best.pt  --data /content/cat/cat.yaml \
                           --project /content/drive/MyDrive/Colab\ Notebooks/cat_dataset/ultra_workdir_yolov5/output --name=test_result --exist-ok --img 640 --iou 0.65

val: data=/content/cat/cat.yaml, weights=['/content/ultra_workdir/cat/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/Colab Notebooks/cat_dataset/ultra_workdir_yolov5/output, name=test_result, exist_ok=True, half=False, dnn=False
YOLOv5 🚀 v6.1-380-gb551098 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 367 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs
val: Scanning '/content/cat/labels/val.cache' images and labels... 1878 found, 0 missing, 0 empty, 0 corrupt: 100% 1878/1878 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 59/59 [01:10<00:00,  1.19s/it]
                 all       1878       2603      0.989      0.998      0.994      0.936
               Cat__       

In [16]:
!cd /content/yolov5;python detect.py --source /content/cat2.mp4 \
                            --weights /content/ultra_workdir/cat/weights/best.pt --conf 0.3 \
                            --project=/content/drive/MyDrive/Colab\ Notebooks/cat_dataset/ultra_workdir_yolov5/output --name=run_image --exist-ok --line-thickness 2

detect: weights=['/content/ultra_workdir/cat/weights/best.pt'], source=/content/cat2.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/Colab Notebooks/cat_dataset/ultra_workdir_yolov5/output, name=run_image, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-380-gb551098 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 367 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs
video 1/1 (1/3601) /content/cat2.mp4: 384x640 1 Cat___, Done. (0.034s)
video 1/1 (2/3601) /content/cat2.mp4: 384x640 1 Cat___, Done. (0.034s)
video 1/1 (3/3601) /content/cat2.mp4: 384x640 1 Cat___, Done. (0.034s)
video 1/1 (4/3601) /content/cat2.mp4: 384x640 1 Cat___, Done. (0

In [17]:
!cd /content/yolov5;python detect.py --source /content/cat3.mp4 \
                            --weights /content/ultra_workdir/cat/weights/best.pt --conf 0.3 \
                            --project=/content/drive/MyDrive/Colab\ Notebooks/cat_dataset/ultra_workdir_yolov5/output --name=run_image --exist-ok --line-thickness 2

detect: weights=['/content/ultra_workdir/cat/weights/best.pt'], source=/content/cat3.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/Colab Notebooks/cat_dataset/ultra_workdir_yolov5/output, name=run_image, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-380-gb551098 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 367 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs
video 1/1 (1/4776) /content/cat3.mp4: 384x640 Done. (0.035s)
video 1/1 (2/4776) /content/cat3.mp4: 384x640 Done. (0.034s)
video 1/1 (3/4776) /content/cat3.mp4: 384x640 1 Cat___, Done. (0.034s)
video 1/1 (4/4776) /content/cat3.mp4: 384x640 1 Cat___, Done. (0.027s)
video 1/1 (5/